# Generate exercises using a large language model (LLM)
This is an implementation of the proposal https://miro.com/app/board/uXjVMuVH2kA=/ made by the author unit.

## LLM setup
We use OpenAI for now because we have not yet investigated other options. Might want to do that, though.

In [1]:
!pip install python-dotenv
!pip install openai

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

## Prompt setup
First we define some enums as intended by the prototype:

In [3]:
from enum import Enum

class Difficulty(Enum):
    LOW: str = "leicht"
    MEDIUM: str = "moderat"
    HIGH: str = "knifflig"

class ExerciseCategory(Enum):
    SINGLE: str = "eine Einzelaufgabe"
    QUIZ: str = "ein Quiz"
    TEST: str = "einen Test"

class ExerciseType(Enum):
    MULTIPLE_CHOICE: str = "Multiple Choice"
    SINGLE_CHOICE: str = "Single Choice"
    GAP_TEXT: str = "Lückentext"
    TRUE_FALSE: str = "Wahr Falsch"
    MAPPING: str = "Zuordnung"
    FREE_TEXT: str = "Freitext"
    FACTUAL_TASK: str = "Sachaufgabe"
    SINGLE_WORD_SOLUTION: str = "Lösung mit 1 Wort"
    SINGLE_NUMBER_SOLUTION: str = "Lösung mit 1 Zahl"

Now we set the actual variables that the frontend would gather, feel free to change the values and experiment:

In [4]:
subject: str = "Mathe"
grade: int = 8
level: Difficulty= Difficulty.MEDIUM
topic: str = "Bruchrechnung"
goal: str = "Die Schüler*innen können Brüche erweitern und kürzen."
category: ExerciseCategory = ExerciseCategory.QUIZ
exercise_types: list[ExerciseType] = [ExerciseType.MULTIPLE_CHOICE, ExerciseType.GAP_TEXT, ExerciseType.MAPPING] 
number_exercises: int = 10
info: str = "Das Quiz wird im Rahmen des Unterrichts an einer Mittelschule eingesetzt. Die Schülerinnen kennen die Grundlagen der Bruchrechnung."

We use the framework langchain because it is convenient, though tasks of this level of complexity could also be done without.

In [5]:
!pip install langchain

Defaulting to user installation because normal site-packages is not writeable


In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
chat = ChatOpenAI(temperature=0.0)
template_string = """
Erstelle {number_exercises} Aufgaben \
für {category} \
in {subject} \
in der Jahrgangsstufe {grade}. \
Der Schwierigkeitsgrad soll {level} sein. \
Das Ziel der Aufgaben ist: {goal} \
Folgende Aufgabentypen sollen hierbei enthalten sein: {exercise_types}. \
{info}
"""
prompt_template = ChatPromptTemplate.from_template(template_string)
prompt_to_generate_exercises = prompt_template.format_messages(subject=subject, 
                                                               grade=grade, 
                                                               level=level.value, 
                                                               topic=topic, 
                                                               goal=goal, 
                                                               category=category.value, 
                                                               exercise_types=', '.join([item.value for item in exercise_types]), 
                                                               number_exercises=number_exercises, 
                                                               info=info
                                                              )
print(prompt_to_generate_exercises[0].content)


Erstelle 10 Aufgaben für ein Quiz in Mathe in der Jahrgangsstufe 8. Der Schwierigkeitsgrad soll moderat sein. Das Ziel der Aufgaben ist: Die Schüler*innen können Brüche erweitern und kürzen. Folgende Aufgabentypen sollen hierbei enthalten sein: Multiple Choice, Lückentext, Zuordnung. Das Quiz wird im Rahmen des Unterrichts an einer Mittelschule eingesetzt. Die Schülerinnen kennen die Grundlagen der Bruchrechnung.



## Generate exercises
Now we see in what this prompt results:

In [7]:
generated_exercises = chat(prompt_to_generate_exercises)
print(generated_exercises.content)

1. Multiple Choice: Welcher Bruch ist gleichwertig zu 3/4?
a) 2/3
b) 4/5
c) 6/8
d) 5/6

2. Lückentext: Kürze den Bruch 12/18 auf den kleinstmöglichen Bruch. Das Ergebnis lautet _______.

3. Zuordnung: Ordne die Brüche ihren gekürzten Formen zu:
a) 8/12
b) 5/10
c) 9/15

4. Multiple Choice: Welcher Bruch ist größer als 1/2?
a) 2/3
b) 3/4
c) 1/3
d) 1/4

5. Lückentext: Erweitere den Bruch 2/5 auf den Bruch mit dem Nenner 10. Das Ergebnis lautet _______.

6. Zuordnung: Ordne die Brüche ihren erweiterten Formen zu:
a) 3/7
b) 4/9
c) 2/5

7. Multiple Choice: Welcher Bruch ist gleichwertig zu 6/8?
a) 3/4
b) 2/3
c) 4/5
d) 5/6

8. Lückentext: Kürze den Bruch 16/24 auf den kleinstmöglichen Bruch. Das Ergebnis lautet _______.

9. Zuordnung: Ordne die Brüche ihren gekürzten Formen zu:
a) 10/15
b) 6/9
c) 12/18

10. Multiple Choice: Welcher Bruch ist kleiner als 3/4?
a) 2/3
b) 4/5
c) 5/6
d) 1/2
